<a href="https://colab.research.google.com/github/JasonLi14/Discontinuity-Finder/blob/main/Test_Symbolic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sympy import *
import math
import queue
import warnings
from typing import Any, Self

# Discontinuity Detector
## Introduction
This is a Jupyter notebook testing some functions with Python's SymPy library to symbolically find and categorize discontinuities.

This program currently works similarly to human intuition. It detects points that could contain a discontinuity and then evaluates the function and limit at those points and categorizes them. While SymPy supports evaluating functions and limits, this notebook seeks to create a method to find these "problem points".

The main idea is from this fact: If $f$ is continuous at $c$ and $g$ is continuous at $f(c)$, then $g \circ f$ is continuous at $c$.

To see this, since $g$ is continuous at $f(c)$, for all $\epsilon > 0$ there exists $\delta_1 > 0$ such that if $|f(c) - x| < \delta_1$ then $|g(f(c)) - g(x)| < \epsilon$. Likewise, since $f$ is continuous at $c$, there exists $\delta_2$ such that $|c-x| < \delta_2$ implies $|f(c)-f(x)| < \delta_2$. Therefore, if $|c-x| < \delta_2$, we have that $|f(c)-f(x)| < \delta_1$ and thus $|g(f(c)) - g(f(x))| < \epsilon$, i.e. $\lim_{x \rightarrow c} g(f(x)) = g(f(c))$, as required. $\square$

The same property applies to addition, multiplication, and subtraction.

We can exploit this fact by parsing the entire function as a series of compositions. We disregard all points we know that are continuous and only focus on the "problem points." Then, we evaluate the function and sided limits at these "problem points" to categorize them as various discontinuities.

## Input
To start, let's input and parse the function as an expression tree that we can later traverse.

In [2]:
# Initialize x as the symbol we use
x = symbols('x')

def getInput(str_input: str = None) -> Expr:
  # Require that str_input is an expression in terms of x

  # We can also supply this input function with a string
  if str_input is None:
    func = input("f(x) = ")
  else:
    func = str_input
  # Parse the input
  parsed_func = parsing.sympy_parser.parse_expr(func,
                                                transformations='all',
                                                evaluate=false)
  # Turn the expression into a tree to recurse through.
  func_tree = srepr(parsed_func)
  # Display the results
  print("Input: ")
  display(parsed_func)
  return parsed_func

# Examples
getInput("3x/28")
getInput("4x**32")
getInput("e**x")
getInput("sin(pi * x)")


Input: 


3*x/28

Input: 


4*x**32

Input: 


e**x

Input: 


sin(pi*x)

sin(pi*x)

## Accumulating Points to Check
We need to set up how each operation affects the "problem points". We will do this with our set of modify functions, which take in the arguments of the expression and return the set of points that may contain a discontinuity. For instance, when we add or multiply, the problem points are not affected, so we just return both.

In [3]:
# Examples
display(S.EmptySet)
display(FiniteSet(0))
display(FiniteSet(0).union(FiniteSet(1)))

EmptySet

{0}

{0, 1}

What about division, say $\frac{f(x)}{g(x)}$? Actually, SymPy doesn't represent division with a different class. Instead, to resolve division, we actually need to implement the power function. This raises the question: when is $f^g$ continuous? Let's split it into a few cases:
1. $f(c) > 0$ and $f, g$ are continuous at $c$, then $f^g$ is continuous.
2. $f(c) = 0, g(c) > 0$,  and $f, g$ are continuous at $c$, then $f^g$ is continuous.
3. $f(c) = 0, g(c) \leq 0$, then $f^g$ does not exist at $c$ and therefore is not continuous.
4. $f(c) < 0, g(x) = a \in \mathbb{Q}$ for some constant $a$ near $c$, $a$ has odd denominator when reduced, and $f$ is continuous at $c$, then $f^g = f^a$ is continuous at $c$.
5. $f(c) < 0$, $g$ varies (heuristically $g$ depends on $x$) near x, then it's probably discontinuous. The idea is that for all positive $\epsilon$, there exists a rational number in reduced form $\frac{p_1}{q_1}$ in $[g(c) - \epsilon, g(c))$ and $\frac{p_2}{q_2}$ in $(g(c), g(c) + \epsilon]$. Then, there is an odd integer in $[2p_1q_2, 2p_2q_1]$, say $a$, such that $\frac{a}{2q_1q_2} \in [\frac{p_1}{q_1}, \frac{p_2}{q_2}]$. But $f(c)^{\frac{a}{2q_1q_2}}$ does not exist.

In summary, we accumulate problem points as follows:
1. Add points such that $f(c) = 0$ and $g(c) \leq 0$
2. Add points such that $f(c) < 0$ and $g$ depends on $x$.
3. Add points such that $f(c) < 0$ and $g$ is an irrational constant or a fraction with even denominator.
4. Add points where either $f$ or $g$ are discontinuous.

We also add a domain parameter in the case of piecewise functions, which we resolve later.


In [4]:
def powModify(f_exp: Expr, g_exp: Expr,
               curr_problem_points: Set = S.EmptySet,
               domain: Set = S.Reals) -> Set:
  # Case 4
  problem_points = curr_problem_points.intersect(domain)
  # Case 1
  f_zeroes = solveset(Eq(f_exp, 0), x, domain=domain)
  g_negative = solveset(LessThan(g_exp, 0), x, domain=domain)
  # Add them to our problem points
  problem_points = problem_points.union((f_zeroes).intersect(g_negative))
  # Case 2
  # Check if g depends on x
  if x in g_exp.free_symbols:
    f_negative = solveset(StrictLessThan(f_exp, 0), x, domain=domain)
    problem_points = problem_points.union(f_negative)
  # Case 3
  # Check if g is a constant that is a rational number
  else:
    g_value = g_exp.doit()
    # If g is irrational or g has even denominator
    if (g_value not in S.Rationals) or (g_value.q % 2 == 0):
      f_negative = solveset(StrictLessThan(f_exp, 0), x, domain=domain)
      problem_points = problem_points.union(f_negative)

  # Only include points specified in the domain
  return problem_points

# Examples
display(powModify(x, Integer(-1)))
display(powModify(x, Integer(1)))
display(powModify(x, x))
display(powModify(S.Exp1, x))
display(powModify(sin(x), S.Zero))
display(powModify(sin(x), S.Zero, domain=Interval(0, 5)))
display(powModify(x, Rational(1, 2)))
# Note that solveset only returns one interval for sin(x) < 0
display(powModify(sin(x), Rational(1, 2)))
display(powModify(x, x, FiniteSet(1)))

{0}

EmptySet

Interval(-oo, 0)

EmptySet

Union(ImageSet(Lambda(_n, 2*_n*pi), Integers), ImageSet(Lambda(_n, 2*_n*pi + pi), Integers))

{0, pi}

Interval.open(-oo, 0)

Interval.open(pi, 2*pi)

Union({1}, Interval(-oo, 0))

Let's add a few more classes of functions. We implement natural logarithms, which cannot accept zero or negative arguments, but is continuous everywhere else.

In [5]:
def logModify(f_exp: Expr,
               f_problem_point: Set = S.EmptySet,
               domain: Set = S.Reals) -> Set:
  return f_problem_point.union(solveset(LessThan(f_exp, 0),
                                        x, domain=domain))

display(logModify(x + 1))
display(logModify(S.Exp1))
# Notice that sympy will only return one interval
display(logModify(sin(x)))
# Which is in [0, 2pi]. The following doesn't work
display(logModify(sin(x), domain=Interval(190, oo)))

Interval(-oo, -1)

EmptySet

Union({0}, Interval.Ropen(pi, 2*pi))

EmptySet

Sine and cosine are continuous on $\mathbb{R}$. For the other trigonometric functions, which can be represented with division, we use what we have already built:

In [6]:
# For modifying either secant or tangent functions
def secTanModify(f_exp: Expr,
               f_problem_point: Set = S.EmptySet,
               domain: Set = S.Reals) -> Set:
  return powModify(cos(f_exp), Integer(-1), f_problem_point, domain)

# For modifying either cosecant or cotangent functions
def cscCotModify(f_exp: Expr,
               f_problem_point: Set = S.EmptySet,
               domain: Set = S.Reals):
  return powModify(sin(f_exp), Integer(-1), f_problem_point, domain)

# Examples
display(cscCotModify(x))
display(secTanModify(x))
display(cscCotModify(x, FiniteSet(1)))
display(cscCotModify(1))

Union(ImageSet(Lambda(_n, 2*_n*pi), Integers), ImageSet(Lambda(_n, 2*_n*pi + pi), Integers))

Union(ImageSet(Lambda(_n, 2*_n*pi + pi/2), Integers), ImageSet(Lambda(_n, 2*_n*pi + 3*pi/2), Integers))

Union({1}, ImageSet(Lambda(_n, 2*_n*pi), Integers), ImageSet(Lambda(_n, 2*_n*pi + pi), Integers))

EmptySet

The floor and ceiling functions are very interesting functions for this project as they have many jump discontinuities, so I would like to implement them. For these, we look for where $f(x)$ is an integer, the idea being that possibly $⌊f(x - \delta)⌋ \neq ⌊f(x)⌋$ or $⌊f(x)⌋ \neq ⌊f(x + \delta)⌋$. If $f(c)$ is not an integer and $f$ is continuous at $c$, then there is a $\delta$ small enough such that $⌊f(c - \delta)⌋ = ⌊f(c)⌋ = ⌊f(c + \delta)⌋$.

To make this easier to find solutions, I will impose a restriction that Sympy can solve $f(y) = x$ for given $x$. That way, we can use an image set $\{f^{-1}(x) ~|~ x \in \mathbb{Z}\}$ to denote the points to include.

In [7]:
def floorCeilModify(f_exp: Expr, f_problem_point: Set = S.EmptySet,
                      domain: Set = S.Reals) -> Set:
  # Find the inverse of f_exp
  y = Symbol('y')
  f_invs = solve(f_exp - y, x)
  problem_points = f_problem_point
  for inv in f_invs:
    # Replace free symbols y with x to keep consistent
    inv = inv.subs(y, x)
    # Ensure that it stays on the real line
    if not inv.has(S.ImaginaryUnit):
      problem_points = problem_points.union(ImageSet(Lambda(x, inv),
                                            Intersection(S.Integers,
                                                        domain)))
  return problem_points
display(floorCeilModify(x))
display(floorCeilModify(x**2))
display(floorCeilModify(x**3))

Integers

Union(ImageSet(Lambda(x, -sqrt(x)), Integers), ImageSet(Lambda(x, sqrt(x)), Integers))

ImageSet(Lambda(x, x**(1/3)), Integers)

Now that we have a few functions to work with, let's create a function to accumulate all the problem points in an expression tree while we traverse the tree. The idea is that for functions $f$ and $g$, for $f(g(x))$ we find where $g$ may be discontinuous, then solve for potential discontinuities of $f \circ g$, and union those points together.

In [8]:
# Accumulates problem points
def accProblemPoints(expr: Expr, domain: Set = S.Reals) -> Set:
  problem_points = S.EmptySet
  # Iterate through the arguments and recurse
  for arg in expr.args:
    problem_points = problem_points.union(accProblemPoints(arg))
  # Add new problem points depending on the type of the argument
  expr_class = expr.func
  if expr.func == Pow: # Powers
    problem_points = powModify(expr.args[0], expr.args[1],
                                problem_points, domain)
  elif expr.func == log: # Logarithms
    problem_points = logModify(expr.args[0], problem_points, domain)
  elif expr.func == csc or expr.func == cot: # Cosecants, cotangents
    problem_points = cscCotModify(expr.args[0], problem_points, domain)
  elif expr.func == sec or expr.func == tan:  # Secants, tangents
    problem_points = secTanModify(expr.args[0], problem_points, domain)
  elif expr.func == floor or expr.func == ceiling: # Floors, ceilings
    problem_points = floorCeilModify(expr.args[0], problem_points, domain)
  elif expr.func in (Abs, Add, Mul, sin, cos, Symbol) or expr.is_number:
    pass  # Do nothing
  else:
      # Other functions are not supported (yet)
      warnings.warn(f"{expr.func} is not yet supported.")
  return problem_points

# Examples
display(accProblemPoints(x + 2))
display(accProblemPoints(1 / x))
display(accProblemPoints(Mul(x, Pow(x, -1), evaluate=false)))
display(accProblemPoints(log(x)))
display(accProblemPoints(1/(sin(x))))
display(accProblemPoints(1/(x**2 + 4 * x + 4)))
display(accProblemPoints(cot(3 * x)))
display(accProblemPoints(1/((1/x)**2 + 4 * (1/x) + 4)))
display(accProblemPoints(1/((1/x)**2 + 4 * (1/x) + 4)))
display(accProblemPoints(floor(x/2)))

EmptySet

{0}

{0}

Interval(-oo, 0)

Union(ImageSet(Lambda(_n, 2*_n*pi), Integers), ImageSet(Lambda(_n, 2*_n*pi + pi), Integers))

{-2}

Union(ImageSet(Lambda(_n, 2*_n*pi/3), Integers), ImageSet(Lambda(_n, 2*_n*pi/3 + pi/3), Integers))

{-1/2, 0}

{-1/2, 0}

ImageSet(Lambda(x, 2*x), Integers)

## Classifying Discontinuities
After accumulating problem points, we can evaluate the limit and the function at these points to categorize the discontinuity. The following function takes in a single value and a funtion and classifies the discontinuity among the following:
1. Continuous, i.e. $f(c) = \lim_{x \rightarrow c} f(x)$
2. Removable discontinuity, i.e. $f(c)$ and $\lim_{x \rightarrow c} f(x)$ exist but $f(c) \neq f(x)$
3. Jump discontinuity, i.e. $\lim_{x \rightarrow c^+} f(x) \neq \lim_{x \rightarrow c^-} f(x)$
4. Essential discontinuity, where at least one of $\lim_{x \rightarrow c^+} f(x), \lim_{x \rightarrow c^-} f(x)$ do not exist.
5. Infinite discontinuity, where at least one of $\lim_{x \rightarrow c^+} f(x), \lim_{x \rightarrow c^-} f(x)$ evaluate to positive or negative infinity.

In [9]:
def classifyVals(f_val: Number, f_limit_l: Number | AccumBounds,
                  f_limit_r: Number | AccumBounds):
  # Check if both limits are numbers (including infinity)
  if f_limit_l.is_number or f_limit_r.is_number:
    if f_limit_l.is_infinite or f_limit_l.is_infinite:
      # Infinite discontinuity
      return "Infinite Discontinuity"
    # Check if they are equal
    elif f_limit_l == f_limit_r:
      # Check if it is equal to f
      if f_limit_l == f_val:
        return "Continuous"
      else: # They are not equal
        return "Removable Discontinuity"
    else: # Otherwise, it is a jump discontinuity
      return "Jump Discontinuity"
  else:  # One of the sided limits DNE, so Essential Discontinuity
    return "Essential Discontinuity"


def classify(f_exp: Expr, val: Number, domain: Set = S.Reals) -> str:
  # Evaluate f at val
  f_val = f_exp.subs(x, val, simultaneous=True)
  # Evaluate the sided limits limit
  f_limit_l = limit(f_exp, x, val, '-')
  f_limit_r = limit(f_exp, x, val, '+')
  # Check if the value is even in the domain specified
  if not val in domain:
    return "Not in Domain"
  else:
    return classifyVals(f_val, f_limit_l, f_limit_r)


# Examples
print(classify(1/x, 0))
print(classify(S.Exp1**(1/x), 0))
print(classify(1/x, 1))
print(classify(Abs(x)/x, 0))
print(classify(Abs(x)/x, 1))
print(classify(sin(x)/x, 0))
print(classify(sin(1/x), 0))
print(classify((x + 2)/(x**2 + 4 * x + 4), -2))
# It seems like Sympy doesn't evaluate the following as non-real:
print(Pow(Pow(x, -1), -1, evaluate=False).subs(x, 0, simultaenous=True))
# Leading to incorrect classification:
print(classify(Pow(Pow(x, -1), -1, evaluate=False), 0))



Infinite Discontinuity
Jump Discontinuity
Continuous
Jump Discontinuity
Continuous
Removable Discontinuity
Essential Discontinuity
Infinite Discontinuity
0
Continuous


## Iterating through Points to Check
Now, we iterate through the set of problem points and check them. We will put a max on 1000 points checked, which should be enough for most relevant functions. It is mainly to deal with Image Sets to prevent the function from running indefinitely.

Finite sets are easy to deal with. To treat intervals, so far, the implementation for finding problem points is very similar to finding the domain. Therefore, we only consider the boundaries for intervals. For Image Sets, we turn them into iterable objects and cycle through each image set for additional points. We will not support the rationals or reals currently.

Our first function is to create a queue given the

In [10]:

def queueProblemPoints(p_set: Set, max_points: int=1000) -> queue.Queue:
  # Set an accumlator for the points checked
  points_checked = 0
  # Initialize queues
  queued_points = queue.Queue(maxsize=max_points)
  check_next = queue.Queue()
  iterable_sets = []
  check_next.put(p_set)
  # Iterate until we hit the max points or until we've
  # Iterated through all the sets
  while points_checked < max_points and not check_next.empty():
    # Get the first item in check_next
    next_set = check_next.get()
    # Handle each type of set accordingly
    if next_set.func == FiniteSet:
      # For finite sets, add each point to the queue, assuming only
      # discrete values are in the set
      for arg in next_set.args:
        if arg in S.Reals:
          queued_points.put(arg)
          points_checked += 1
    elif next_set.func == Union:
      # For unions, add each argument back to the queue to check
      for arg in next_set.args:
        check_next.put(arg)
    elif next_set.func == Interval:
      # Add the endpoints if unbounded
      if not next_set.is_left_unbounded:
        queued_points.put(next_set.start)
        points_checked += 1
      if not next_set.is_right_unbounded:
        queued_points.put(next_set.end)
        points_checked += 1
    elif next_set.is_iterable:
      # Put the image set/other iterable sets into a new list if iterable,
      iterable_sets.append(iter(next_set))
    elif next_set == S.EmptySet:
      # Don't do anything for empty sets
      pass
    else:
      # Other sets, ex. Condition Sets and Intersects,
      # are not supported currently
      warnings.warn(f"The set {next_set.func} is not yet supported.")


  # Iterate through the image sets
  iterable_sets_len = len(iterable_sets)
  if iterable_sets_len > 0:
    # Keep track of which image set we are on
    image_set_i = 0
    num = 0
    while points_checked < max_points:
      # Put in the next element
      next_point = next(iterable_sets[image_set_i])
      if next_point in S.Reals:
        queued_points.put(next_point)
      points_checked += 1
      # Go check the next num if possible
      if image_set_i + 1 % iterable_sets_len == 0:
        num += 1
      # Rotate through the image sets
      image_set_i = (image_set_i + 1) % iterable_sets_len

  return queued_points

# Examples
display(queueProblemPoints(FiniteSet(0, 1, 2)).queue)
display(queueProblemPoints(ImageSet(Lambda(x, pi * x),
                                      S.Integers), 10).queue)
display(queueProblemPoints(Union(FiniteSet(0), FiniteSet(1))).queue)
display(queueProblemPoints(Union(ImageSet(Lambda(x, 2 * pi * x),
                                            S.Integers),
                                   ImageSet(Lambda(x, 2 * pi * x + pi),
                                            S.Integers)), 10).queue)
display(queueProblemPoints(Interval(-oo, 1)).queue)
display(queueProblemPoints(Interval(2, oo)).queue)
display(queueProblemPoints(Interval(2, 3)).queue)

deque([0, 1, 2])

deque([0, pi, -pi, 2*pi, -2*pi, 3*pi, -3*pi, 4*pi, -4*pi, 5*pi])

deque([0, 1])

deque([0, pi, 2*pi, 3*pi, -2*pi, -pi, 4*pi, 5*pi, -4*pi, -3*pi])

deque([1])

deque([2])

deque([2, 3])

Now, let's check each element in the queue.

In [11]:
def checkProblemPoints(p_queue: queue.Queue, f_exp: Expr,
                         domain: Set = S.Reals) -> list:
  discontinuities = []
  # Iterate through
  while not p_queue.empty():
    next_point = p_queue.get()
    discontinuity_type = classify(f_exp, next_point, domain)
    if not discontinuity_type in ("Continuous", "Not in Domain"):
      discontinuities.append((next_point, discontinuity_type))
  return discontinuities

# Examples
test_queue = queue.Queue()
test_queue.put(0)
display(checkProblemPoints(test_queue, sin(x) / x))
test_queue.put(0)
display(checkProblemPoints(test_queue, sin(1 / x)))
test_queue.put(-1)
display(checkProblemPoints(test_queue, sin(1 / x)))
test_queue.put(0)
test_queue.put(1)
display(checkProblemPoints(test_queue,
                             sin(1 / x) + Abs(x - 1) / (x - 1)))

[(0, 'Removable Discontinuity')]

[(0, 'Essential Discontinuity')]

[]

[(0, 'Essential Discontinuity'), (1, 'Jump Discontinuity')]

## Implementing Piecewise Functions
Common examples of discontinuities arise from piecewise functions. For example, there is a jump discontinuity at $x=1$ for:
$$f(x) = \begin{cases} 0 \text{ if } x < 1 \\ 1 \text{ if } x \geq 1 \end{cases}$$
We are going to restrict piecewise functions to be piecewise on intervals, i.e. no Thomae's function.

Note that there seems to be a bug in Sympy which prevents correct evaluation of sided limits of piecewise functions, which causses a few modifications in the following function. This is why I have opted to write a separate function specifically for piecewise functions rather than implementing it similar to the functions before. It still uses many of the functions and steps that we will see in `detect_discontinuities`, but adapted for piecewise functions.

In [12]:
def detect_discontinuities_piecewise(
    piecewise_func: Expr, max_check: int = 1000):
  # Initialize to accumulate
  problem_points = S.EmptySet
  remaining_pieces = S.Reals

  # Because of the bug, we calculate the limits per piece rather than
  # Of the whole function. The following dicts keep track of the sided
  # limits.
  left_limits = {}
  right_limits = {}

  # Check if it is a piecewise function
  if not piecewise_func.is_Piecewise:
    warnings.warn("The function is not a piecewise function.")
    return
  # Iterate through each piece
  for piece in piecewise_func.args:
    # piece[0] is the function, piece[1] is the condition
    # Find the domain as an interval
    if piece[1] == True:  # The "else," which is at the end
      piece_domain = remaining_pieces
    else:
      if piece[1].func == And:
        # For and, we just intersect
        piece_domain = S.Reals
        for cond in piece[1].args:
          piece_domain = piece_domain.intersect(solveset(cond,
                                                         x, S.Reals))
      elif piece[1].func == Or:
        # For or, we just union
        piece_domain = S.EmptySet
        for cond in piece[1].args:
          piece_domain = piece_domain.union(solveset(cond,
                                                     x, S.Reals))
      else:
        piece_domain = solveset(piece[1], x, S.Reals)
      remaining_pieces = Complement(remaining_pieces, piece_domain)
    # Get the separate pieces (ex. if the condition is x**2 > 4)
    if piece_domain.func == Union:
      piece_domain_list = piece_domain.args
    # Intervals and finite sets we add the entire thing
    elif piece_domain.func == Interval or piece_domain.func == FiniteSet:
      piece_domain_list = [piece_domain]
    else:
      raise ValueError("The condition must create intervals!")

    # Iterate through
    for part in piece_domain_list:
      if part.func == Interval:
        # Accumulate the end points
        if not part.is_left_unbounded:
          problem_points = problem_points.union(FiniteSet(part.start))
          left_limits[part.start] = limit(piece[0], x, part.start, '+')
        if not part.is_right_unbounded:
          problem_points = problem_points.union(FiniteSet(part.end))
          right_limits[part.end] = limit(piece[0], x, part.end, '-')
      elif piece_domain.func == FiniteSet:
        # Just check if those points are problematic
        problem_points = problem_points.union(piece_domain)
      else:
        raise ValueError("The condition must create intervals!")

  # Turn them into a queue
  queued_points = queueProblemPoints(problem_points, max_check)
  # Our answer
  discontinuities = []

  # Iterate through
  while not queued_points.empty():
    next_point = queued_points.get()
    # Find the value and limits
    f_val = piecewise_func.subs(x, next_point)
    # Use our dictionary first when finding sided limits
    if next_point in left_limits.keys():
      f_limit_l = left_limits[next_point]
    else:
      f_limit_l = limit(piecewise_func, x, next_point, '-')
    if next_point in right_limits.keys():
      f_limit_r = right_limits[next_point]
    else:
      f_limit_r = limit(piecewise_func, x, next_point, '-')
    discontinuity_type = classifyVals(f_val, f_limit_l, f_limit_r)
    if not discontinuity_type in ("Continuous", "Not in Domain"):
      discontinuities.append((next_point, discontinuity_type))
  return discontinuities

# Examples
display(detect_discontinuities_piecewise(Piecewise((0, x < 1),
                                                   (1, x > 1),
                                                   (2, True))))
display(detect_discontinuities_piecewise(Piecewise((0, x <= 1),
                                                   (1, x > 1),
                                                   (2, True))))
display(detect_discontinuities_piecewise(Piecewise((x + 1, x <= 0),
                                                   (x - 1, x > 0),
                                                   (2, True))))
display(detect_discontinuities_piecewise(Piecewise((x**2/x**3,
                                                    And(x > 5, x < 9)),
                                                   (2, True))))
display(detect_discontinuities_piecewise(Piecewise((x, Ne(x, 0)),
                                                   (2, True))))
display(detect_discontinuities_piecewise(Piecewise((x / x, Ne(x, 0)),
                                                   (1, True),
                                                   evaluate=False)))

[(1, 'Jump Discontinuity')]

[(1, 'Jump Discontinuity')]

[(0, 'Jump Discontinuity')]

[(5, 'Jump Discontinuity'), (9, 'Jump Discontinuity')]

[(0, 'Removable Discontinuity')]

[]

[]

[(0, 'Removable Discontinuity')]

## Putting Everything Together
So we have all the functions to classify discontinuities. Let's combine them all, starting from input and ending with the list of discontinuities.

In [20]:
def detect_discontinuities(input_str: str = None, max_check: int = 1000):
  # Get input
  parsed_f = getInput(input_str)
  # Use our special function if piecewise
  if parsed_f.func == Piecewise:
    return detect_discontinuities_piecewise(parsed_f, max_check)
  else:
    # Accumulate problem points
    problem_points = accProblemPoints(parsed_f)
    # Turn them into a queue
    queued_points = queueProblemPoints(problem_points, max_check)
    # Check problem points
    discontinuities = checkProblemPoints(queued_points, parsed_f)
    return discontinuities

# Examples
print(detect_discontinuities("1/sin(x)", 5))
print(detect_discontinuities("sqrt(x)", 5))
print(detect_discontinuities("tan(x)", 5))
print(detect_discontinuities("tan(x)cos(x)", 5))
print(detect_discontinuities("(x+1)(x+2)/((x+1))"))
print(detect_discontinuities("x/sin(x)", 5))
print(detect_discontinuities("floor(x)", 5))
print(detect_discontinuities("abs((1/x)+2)/((1/x)+2)"))
piecewise_1 = "Piecewise((sin(x) / x, Ne(x, 0)), (1, True))"
print(detect_discontinuities(piecewise_1))
piecewise_2 = "Piecewise((sin(x) / x, Ne(x, 0)), (2, True))"
print(detect_discontinuities(piecewise_2))


Input: 


1/sin(x)

[(0, 'Infinite Discontinuity'), (pi, 'Infinite Discontinuity'), (2*pi, 'Infinite Discontinuity'), (3*pi, 'Infinite Discontinuity'), (-2*pi, 'Infinite Discontinuity')]
Input: 


sqrt(x)

[]
Input: 


...

[(-1/2, 'Jump Discontinuity'), (0, 'Jump Discontinuity')]
Input: 


Piecewise((sin(x)/x, Ne(x, 0)), (1, True))

[]
Input: 


Piecewise((sin(x)/x, Ne(x, 0)), (2, True))

[(0, 'Removable Discontinuity')]


This cell will allow input:

In [14]:
detect_discontinuities()

f(x) = sin(1/4x)
Input: 


sin(1*x/4)

[]

## Conclusion
In this notebook, we have created functions to detect discontinuities by viewing the function as a chain of function compositions, and propogate points we suspect are discontinuous backwards. The reason I chose this approach is because there are not many properties that discontinuous functions share, specifically functions with removable discontinuities that I sought to detect. For example, we can create a function discontinuous at $x=c$ by introducing a removable discontinuity at $c$. For this reason, I could not think of any ways I could exploit properties of continuous functions to find discontinuities other than a thorough search method implemented here. However, this method relies heavily on some Sympy operations, in particular the set operations, and as a result, may be inefficient. After comparing to Sympy's implementation of the `continuous_domain` and `singularities` method, the two methods seem to have the same essence, except Sympy does not take a recursive approach. Instead, it iterates through the `atoms` of the expression to find points not in the domain, which is probably more efficient.

The largest improvement I would like to implement, outside of supporting more functions, is a more elegant way of dealing with image sets. Currently, for a function like $\frac{1}{\sin(x)}$, the program gets an image set of all $x$ such that $\sin(x) = 0$, then testing each point in the image set individually. I wonder if it is possible to implement some heuristics for the program to instead return an image set describing $x$ such that $f$ has a specific type of discontinuity at $x$.
